In [1]:
import pandas as pd
from xml.dom import minidom
import pickle
import os
import glob

In [2]:
kadaster_vbObject = "kadaster_vbObject"
columns = ["vblObject","gebruiksdoel","oppervlakte","vbloStatus"]
if os.path.isfile(kadaster_vbObject + ".csv"):
    kadaster = pd.read_csv(kadaster_vbObject + ".csv")
else: kadaster = pd.DataFrame(columns=columns)

checkedVBO = "checkedVBOfiles"
if os.path.isfile(checkedVBO + ".p"):
    with open(checkedVBO + ".p", 'rb') as fp:
        checkedVBOfiles = pickle.load(fp)
else: checkedVBOfiles = []
    
kadaster_ids = pd.read_csv("kadaster_ids.csv", dtype = object)

In [3]:
checkedVBOfiles

[]

In [4]:
kadaster_ids.head()

,huisnummer,postcode,nrAanduiding,vblObject,inOnderzoek
0,9,2984BG,0597200000261896,0597010000261897,False
1,1,8911AW,0080200000419089,0080010000419090,False
2,2,8448CN,0074200000373763,0074010000373764,False
3,3,8801JK,0070200000332718,0070010000332719,False
4,12,9061AE,0737200000037159,0737010000005537,False


In [5]:
len(kadaster)

0

In [9]:
prefix = "kadaster/9999VBO08112019/"

In [10]:
cut = len(prefix)

In [11]:
files = glob.glob(prefix+"*xml")

In [12]:
files

['kadaster/9999VBO08112019/9999VBO08112019-000001.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000002.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000003.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000004.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000005.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000006.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000007.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000008.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000009.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000010.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000011.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000012.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000013.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000014.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000015.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000016.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-000017.xml',
 'kadaster/9999VBO08112019/9999VBO08112019-00001

In [13]:
i = iter(files)

In [14]:
currentfile = next(i)
while currentfile in checkedVBOfiles:
    print(currentfile,"\talready checked")
    currentfile = next(i)
print(currentfile)

kadaster/9999VBO08112019/9999VBO08112019-000001.xml


In [15]:
currentfile[cut:]

'9999VBO08112019-000001.xml'

In [35]:
def checkfile(currentfile):
    mydoc = minidom.parse(currentfile)
    items = mydoc.getElementsByTagName("bag_LVC:Verblijfsobject")
    for item in items[:1]:
        for child in item.childNodes:
            if child.nodeName[8:] == 'identificatie':
                id = child.firstChild.nodeValue
            elif child.nodeName[8:] == 'gebruiksdoelVerblijfsobject':
                gebruiksdoel = child.firstChild.nodeValue
            elif child.nodeName[8:] == 'oppervlakteVerblijfsobject':
                oppervlakte = child.firstChild.nodeValue
            elif child.nodeName[8:] == 'verblijfsobjectStatus':
                status = child.firstChild.nodeValue[16:]
        if kadaster_ids.where((kadaster_ids["vblObject"] == id)).dropna().empty == False:
#             print(id,gebruiksdoel,oppervlakte,status)
            vbObject = pd.DataFrame(data = [[id,gebruiksdoel,oppervlakte,status]], columns=columns)
            return vbObject
        else:
            return None

In [37]:
vbObject = checkfile(currentfile)
print(vbObject == None)

True


In [38]:
print(vbObject != None)

False


## Get gebruiksdoel

In [92]:
%%time
for item in items[:1]:
    for child in item.childNodes:
#         print(child.nodeName[8:], child.firstChild.nodeValue)
        if child.nodeName[8:] == 'identificatie':
            id = child.firstChild.nodeValue
        elif child.nodeName[8:] == 'gebruiksdoelVerblijfsobject':
            gebruiksdoel = child.firstChild.nodeValue
        elif child.nodeName[8:] == 'oppervlakteVerblijfsobject':
            oppervlakte = child.firstChild.nodeValue
        elif child.nodeName[8:] == 'verblijfsobjectStatus':
            status = child.firstChild.nodeValue[16:]
    if kadaster_ids.where((kadaster_ids["vblObject"] == id)).dropna().empty == False:
#         print(id,gebruiksdoel,oppervlakte,status)
        vbObject = pd.DataFrame(data = [[id,gebruiksdoel,oppervlakte,status]], columns=columns)
        kadaster = kadaster.append(vbObject)

0000010000057469 woonfunctie 72 in gebruik
CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 7.82 ms


In [93]:
kadaster

,vblObject,gebruiksdoel,oppervlakte,vbloStatus
0,0000010000057469,woonfunctie,72,in gebruik


In [27]:
checkedVBOfiles.append(currentfile)
with open(checkedVBO + '.p', 'wb') as fp:
        pickle.dump(checkedVBOfiles, fp)